In [1]:
import pandas as pd
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io 
import os
import warnings
import datetime
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('alert_data.csv')

In [4]:
df.head()

,ts,date,hm,active_users_feed,views,likes,ctr
0,2022-03-29 00:00:00,2022-03-29 00:00:00,00:00,391,8355,8355,22.67
1,2022-03-29 00:15:00,2022-03-29 00:00:00,00:15,402,8558,8558,22.45
2,2022-03-29 00:30:00,2022-03-29 00:00:00,00:30,387,7949,7949,21.86
3,2022-03-29 00:45:00,2022-03-29 00:00:00,00:45,399,8608,8608,22.17
4,2022-03-29 01:00:00,2022-03-29 00:00:00,01:00,366,7951,7951,22.06


In [5]:
metrics = [df.active_users_feed, df.views, df.likes, df.ctr]

In [ ]:
def check_anomaly(m, alpha=3):
    last_value = m.iloc[-1,-1]
    m_last_2hour = m[m.iloc[:, 0] > m.iloc[-1,0] - datetime.timedelta(hours=2)]
    x_mean = np.mean(m_last_2hour.iloc[:-1, -1])
    x1 = x_mean - alpha * np.std(m_last_2hour.iloc[:-1, -1])
    x2 = x_mean + alpha * np.std(m_last_2hour.iloc[:-1, -1])
    x_mean = np.mean(m_last_2hour.iloc[:-1, -1])
    if last_value > x2:
        is_alert = 1
        diff = round(((last_value - x2)/x2)*100, 2)
        return m.columns[-1], is_alert, last_value, diff
    elif last_value < x1:
        is_alert = 1
        diff = round(((last_value - x1)/x1)*100, 2)
        return m.columns[-1], is_alert, last_value, diff
    else:
        is_alert = 0
        return (0, 0, 0, 0) 

In [ ]:
def run_alerts(chat = None):
    chat_id = chat
    bot = telegram.Bot(token=os.environ.get('REPORT_BOT_TOKEN'))
    metric_name = {'active_users_feed' : "'Количество пользователей ленты новостей'", 'views' : "'Количество просмотров'",\
                  'likes': "'Количество лайков'", 'ctr' : "CTR"}
    
    for m in metrics:
        metric, is_alert, last_value, diff = check_anomaly(m, alpha=3)
        if is_alert:
            msg = f'Метрика {metric_name[metric]} вышла за границы доверительного интервала.\nТекущее значение {last_value}.\nОтклонение от ожидаемого значения {diff}%'
             
            last_15sec = m.iloc[-1,0]
            m_last_2hour = m[m.iloc[:, 0] > last_15sec - datetime.timedelta(hours=2)]
            last_15sec_day_ago = last_15sec - pd.DateOffset(days=1)
            m_last_2hour_yesterday = m[(m.iloc[:, 0] > last_15sec_day_ago - datetime.timedelta(hours=2))&(m.iloc[:, 0] < last_15sec_day_ago + datetime.timedelta(hours=2))]
            m['roling_mean'] = m.iloc[:, -1].rolling(8).mean()

            sns.set(rc={'figure.figsize': (16, 10)}) 
            plt.tight_layout()
            
            ax = sns.lineplot(x=m_last_2hour["hm"], y = m_last_2hour[metric])
            ax = sns.lineplot(x=m_last_2hour_yesterday["hm"], y = m_last_2hour_yesterday[metric])
            ax = sns.lineplot(x=m_last_2hour["hm"], y = m['roling_mean'])
            ax.set_title(f'{metric_name[metric]}')      
            ax.legend(['сегодня', 'вчера', 'среднее за последние 2 часа'])
            
            plot_object = io.BytesIO()
            plt.savefig(plot_object) 
            plot_object.name = 'test_plot.png' 
            plot_object.seek(0)
            plt.close()
            
            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.send_photo(chat_id=chat_id, photo=plot_object)

In [ ]:
try:
    run_alerts()
except Exception as e:
    print(e)